# Computation of layer by layer containment corrections




## Setup

In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools


In [3]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v80'
version_V8 = version
version_V9 = 'v96'


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200_calibs', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version_V8, 'V8'),
    Sample('ele_flat2to100_PU200_calibs', 'PU200', version_V8, 'V8')
    ]


samples_ele_V9 = [
    Sample('ele_flat2to100_PU0_calibs', 'PU0', version_V9, 'V9'),
    Sample('ele_flat2to100_PU200_calibs', 'PU200', version_V9, 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0_calibs', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200_calibs', 'PU200', version, 'photon')
    ]

samples_photons_V8 = [
    Sample('photon_flat8to150_PU0_calibs', 'PU0', version_V8, 'V8'),
    Sample('photon_flat8to150_PU200_calibs', 'PU200', version_V8, 'V8')
    ]

samples_photons_V9 = [
    Sample('photon_flat8to150_PU0_calibs', 'PU0', version_V9, 'V9'),
    Sample('photon_flat8to150_PU200_calibs', 'PU200', version_V9, 'V9')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0_calibs', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0_calibs', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200_calibs', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200_calibs', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200_calibs', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200_calibs', 'PU200', version, 'V9')
    ]



gen_part_selections: 14


Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU0_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_ele_flat2to100_PU200_calibs_v96.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_calibs_v80.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_calibs_v96.root does not exist
Err

In [104]:
# %load settings.py
# %load settings.py
# from samples import get_label_dict
import python.selections as selections
import python.collections as collections

import pprint



def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary



# === samples =====================================================
samples = []

# samples = samples_photons_V9
samples += samples_ele_V9


sample = 'V9'

do_rate = False

# === TP ==========================================================
# tps = ['EG', 'TkEG', 'TkEle', 'TkIsoEle']
# tps = ['HMvDR', 'HMvDRcylind10', 'HMvDRcylind5', 'HMvDRcylind2p5']

tps = ['HMvDR', 'HMvDRshape', 'HMvDRshapeDr', 'HMvDRcylind5', 'HMvDRcylind2p5',
       'HMvDRshapeCalib', 'HMvDRshapeDrCalib', 'HMvDRcylind5Calib', 'HMvDRcylind2p5Calib',
       # 'HMvDRshapeCalib1', 'HMvDRcylind10Calib1', 'HMvDRcylind5Calib1', 'HMvDRcylind2p5Calib1'
      ]

# === TP selections ================================================
tp_select = {}
tp_select_default = ['Em', 'all']

tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

for tp in tps:
    if not tp in tp_select.keys():
        tp_select[tp] = tp_select_default

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC', 'GENEtaD']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']



event_manager = collections.EventManager()

all_tpsets = {}
for collection in event_manager.collections:
    all_tpsets[collection.name] = collection.label


# all_tpsets = {'DEF': 'dRC3d',
#               'DEFCalib': 'NNDR Calib v1',
#               'DEFNC': 'dRC3d + new Th',
#               'HMvDR': 'HistoMaxC3d + dR(layer)',
#               'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
#               'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
#               'EG': 'EG',
#               'TkEG': 'TkEG',
#               'TkEle': 'TkEle',
#               'TkIsoEle': 'TkIsoEle',
#               'L1Trk': 'L1Track'}


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections_calib))
# gen_selections.update({'nomatch': ''})


pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


--- TPs: 
[   'HMvDR',
    'HMvDRshape',
    'HMvDRshapeDr',
    'HMvDRcylind5',
    'HMvDRcylind2p5',
    'HMvDRshapeCalib',
    'HMvDRshapeDrCalib',
    'HMvDRcylind5Calib',
    'HMvDRcylind2p5Calib']
--- TP selections:
{   'EG': ['EGq2', 'EGq3'],
    'HMvDR': ['Em', 'all'],
    'HMvDRcylind2p5': ['Em', 'all'],
    'HMvDRcylind2p5Calib': ['Em', 'all'],
    'HMvDRcylind5': ['Em', 'all'],
    'HMvDRcylind5Calib': ['Em', 'all'],
    'HMvDRshape': ['Em', 'all'],
    'HMvDRshapeCalib': ['Em', 'all'],
    'HMvDRshapeDr': ['Em', 'all'],
    'HMvDRshapeDrCalib': ['Em', 'all'],
    'L1Trk': ['all', 'Pt2', 'Pt10'],
    'TkEG': [   'EGq2M1P2',
                'EGq3M1P10',
                'EGq2M1P5',
                'EGq2M1P10',
                'EGq3M1',
                'EGq3',
                'EGq2',
                'EGq3M1P5',
                'EGq3M1P2',
                'EGq2M1'],
    'TkEle': [   'EGq2',
                 'EGq3',
                 'EGq2Iso0p2',
                 'EGq3Iso0p2',
  

In [105]:
tpsets


{'HMvDR': 'HM+dR(layer) Cl3d',
 'HMvDRcylind2p5': 'HM Cylinder 2.5cm',
 'HMvDRcylind2p5Calib': 'HM Cylinder 2.5cm calib.',
 'HMvDRcylind5': 'HM Cylinder 5cm',
 'HMvDRcylind5Calib': 'HM Cylinder 5cm calib.',
 'HMvDRshape': 'HM shape',
 'HMvDRshapeCalib': 'HM shape calib.',
 'HMvDRshapeDr': 'HM #Delta#rho < 0.015',
 'HMvDRshapeDrCalib': 'HM #Delta#rho < 0.015 calib.'}

In [106]:
tpset_selections

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'all': ''}

In [107]:
gen_selections

{'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDHBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaBCDLBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5',
 'GENEtaBCHBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} >= 0.5',
 'GENEtaBCLBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} < 0.5',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaDHBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaDHBremPt10': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=10GeV',
 'GENEtaDHBremPt20': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=20GeV',
 'GENEtaDHBremPt30': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=30GeV',
 'GENEtaDHBremPt40': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=40GeV',
 'GENEtaDLBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5'}

In [108]:
from python.plotters import tp_calib_plotters

histo_primitives = pd.DataFrame() 

for plotter in tp_calib_plotters:
    histo_primitives = histo_primitives.append(plotter.get_histo_primitives(), ignore_index=True)

    
print histo_primitives.data.unique()

['HMvDR' 'HMvDRshape' 'HMvDRshapeDr' 'HMvDRcylind5' 'HMvDRcylind2p5'
 'HMvDRshapeCalib' 'HMvDRshapeDrCalib' 'HMvDRcylind5Calib'
 'HMvDRcylind2p5Calib']


In [109]:
files

{}

In [110]:
class EnergyRes(histos.ResoHistos):
    def __init__(self,  name, root_file=None, debug=False):
        self.name = name
        self.h_energyResVenergy = ROOT.TH2F('{}_energyResVenergy'.format(self.name), '', 100,0,1000, 200, -100, 100)
        histos.BaseResoHistos.__init__(self, name, root_file)
    
    def fill(self, tree, expression):
        hname = '{}_energyResVenergy'.format(self.name)
        draw_expression = '{}>>{}(100,0,1000, 200, -100, 100)'.format(expression, hname)
        tree.Draw(draw_expression,"","COL")
        self.h_energyResVenergy = ROOT.gPad.GetPrimitive(hname)

## Computation of the values

In [111]:
calib_table = {}
# calib_table['HMvDRfixedCalib'] = [4.5, 0.94, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.91, 1.06, 1.07, 1.37, 1.47]
# calib_table['HMvDRcylind10Calib'] = [4.5, 0.94, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.91, 1.06, 1.07, 1.37, 1.47]
# calib_table['HMvDRcylind5Calib'] = [4.91, 0.94, 1.09, 1.05, 0.92, 1.05, 1.11, 0.94, 1.02, 0.92, 1.1, 1.04, 1.34, 1.6]
# calib_table['HMvDRcylind2p5Calib'] = [8.49, 0.97, 1.19, 1.18, 1.04, 1.1, 1.25, 1.05, 1.14, 0.97, 1.17, 1.34, 1.3, 1.67]
#                                      [7.86, 1.1, 1.23, 1.0, 1.18, 1.23, 0.99, 1.15, 1.11, 1.16, 0.98, 1.4, 1.47, 1.45]

# # v89
# calib_table['HMvDRfixedCalib'] = [4.52, 0.93, 1.08, 1.04, 0.91, 1.04, 1.09, 0.91, 1.0, 0.9, 1.07, 1.06, 1.36, 1.47]
# calib_table['HMvDRcylind10Calib'] = [4.53, 0.88, 1.14, 1.0, 0.92, 1.06, 1.07, 0.89, 1.06, 0.83, 1.07, 1.13, 1.34, 1.49]
# calib_table['HMvDRcylind5Calib'] = [4.98, 1.03, 0.98, 1.12, 0.91, 1.05, 1.14, 0.87, 1.02, 0.97, 1.13, 1.01, 1.43, 1.4]
# calib_table['HMvDRcylind2p5Calib'] = [7.86, 1.1, 1.23, 1.0, 1.18, 1.23, 0.99, 1.15, 1.11, 1.16, 0.98, 1.4, 1.47, 1.45]
# calib_table['HMvDRshapeCalib'] = [1]*28

# # v93
# calib_table['HMvDRCalib'] = [6.38, 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81]
# calib_table['HMvDRcylind10Calib'] = [6.39, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [6.92, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [12.35, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [8.43, 1.69, 1.1, 1.2, 0.97, 0.97, 1.13, 1.03, 0.82, 1.0, 0.89, 1.08, 1.55, 1.82]
# calib_table['HMvDRshapeDrCalib'] = [1.]*14

# # v96
# calib_table['HMvDRCalib'] = [6.38, 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81]
# calib_table['HMvDRcylind10Calib'] = [6.39, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
# calib_table['HMvDRcylind5Calib'] = [6.92, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
# calib_table['HMvDRcylind2p5Calib'] = [12.35, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
# calib_table['HMvDRshapeCalib'] = [7.17, 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
# calib_table['HMvDRshapeDrCalib'] = [6.96, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.05, 0.83, 1.03, 0.91, 1.07, 1.51, 1.89]

# v96bis
calib_table['HMvDRCalib'] = [1., 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81]
calib_table['HMvDRcylind10Calib'] = [1., 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5, 1.83]
calib_table['HMvDRcylind5Calib'] = [1., 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89]
calib_table['HMvDRcylind2p5Calib'] = [1., 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15]
calib_table['HMvDRshapeCalib'] = [1., 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84]
calib_table['HMvDRshapeDrCalib'] = [1., 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.05, 0.83, 1.03, 0.91, 1.07, 1.51, 1.89]



calib_table['HMvDRfixedCalib1'] = [1.527]+[1.]*12+[1.98]
calib_table['HMvDRcylind10Calib1'] = [1.527]+[1.]*12+[1.98]
calib_table['HMvDRcylind5Calib1'] = [1.527]+[1.]*12+[1.98]
calib_table['HMvDRcylind2p5Calib1'] = [1.527]+[1.]*12+[1.98]
calib_table['HMvDRshapeCalib1'] = [1.527]+[1.]*12+[1.98]

print calib_table

{'HMvDRCalib': [1.0, 0.98, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.81, 1.0, 0.9, 1.08, 1.5, 1.81], 'HMvDRcylind5Calib': [1.0, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.06, 0.83, 1.04, 0.92, 1.08, 1.5, 1.89], 'HMvDRcylind10Calib1': [1.527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.98], 'HMvDRfixedCalib1': [1.527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.98], 'HMvDRshapeCalib1': [1.527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.98], 'HMvDRcylind2p5Calib': [1.0, 0.87, 1.16, 1.21, 1.06, 1.04, 1.25, 1.15, 0.97, 1.11, 1.01, 1.15, 1.64, 2.15], 'HMvDRshapeCalib': [1.0, 1.38, 1.05, 1.07, 0.96, 0.97, 1.11, 1.04, 0.81, 1.02, 0.9, 1.07, 1.52, 1.84], 'HMvDRcylind5Calib1': [1.527, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.98], 'HMvDRshapeDrCalib': [1.0, 0.98, 1.05, 1.09, 0.96, 0.97, 1.11, 1.05, 0.83, 1.03, 0.91, 1.07, 1.51, 1.89], 'HMvDRcylind10Calib': [1.0, 0.99, 1.03, 1.07, 0.94, 0.96, 1.09, 1.03, 0.8, 1.02, 0.9, 1.08, 1.5,

In [112]:
tps

['HMvDR',
 'HMvDRshape',
 'HMvDRshapeDr',
 'HMvDRcylind5',
 'HMvDRcylind2p5',
 'HMvDRshapeCalib',
 'HMvDRshapeDrCalib',
 'HMvDRcylind5Calib',
 'HMvDRcylind2p5Calib']

In [113]:

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)

class HWrapper(object):
    def __init__(self, histo):
        self.histo = histo
    
    def get(self, debug=False):
        return self.histo


from sklearn.linear_model import LinearRegression
import array
import numpy as np 
for tp in tps:
    for gen_sel in ['GENEtaBC']:

#     for gen_sel in gen_selections:
        if gen_sel == 'nomatch':
            continue
        for tp_sel in tp_select[tp]:
            print 'TP: {} sel: {}, gen_sel: {}'.format(tp, tp_sel, gen_sel)
            tree_name = '{}_{}_{}_calib'.format(tp, tp_sel, gen_sel)
#             samples[0].histo_file.ls()
#             print tree_name
            tree = samples[0].histo_file.Get(tree_name)
#             tree.Print()
#             draw([hres.h_energyResVenergy], labels=[''], text='')
            branches = ['e1', 'e3','e5','e7','e9','e11','e13','e15','e17','e19','e21','e23','e25','e27']
            fill_expression = 'e1+e3+e5+e7+e9+e11+e13+e15+e17+e19+e21+e23+e25+e27-Egen:Egen'
            if tp in calib_table.keys():
                calibrated_branches = []
                calib_factors = calib_table[tp]
                fill_expression = ''

                for idx,branch in enumerate(branches):
                    calibrated_branches.append('{}*{}'.format(branch, calib_factors[idx]))
                    if idx != 0:
                       fill_expression += '+' 
                    fill_expression += '{}*{}'.format(branch, calib_factors[idx])
                fill_expression += '-Egen:Egen'
                branches = calibrated_branches
#             print  fill_expression
#             continue
            hres = EnergyRes(name='{}_{}_{}'.format(tp, tp_sel, gen_sel))

            hres.fill(tree, fill_expression)

            layer_energy = rnp.tree2array(tree, branches=branches, cache_size=400000000),
#             print layer_corr
            target_energy = rnp.tree2array(tree, branches='Egen', cache_size=400000000)
#             print layer_energy
            layer_energy_list = [list(aset) for aset in layer_energy[0]]
            if len(layer_energy_list) == 0:
                continue
#             print layer_energy_list
#             print layer_energy_list*layer_corr
            regression = LinearRegression(fit_intercept=False)
            regression.fit(np.array(layer_energy_list), np.array(target_energy))
            print 'Calibration Name: {} nclusters: {}, score: {}, coefficients:'.format(tree_name, len(layer_energy_list), regression.score(np.array(layer_energy_list), np.array(target_energy)))
            print [float("{0:0.2f}".format(i)) for i in regression.coef_ ]
            sumv = 0.
            for value in regression.coef_:
                sumv+=value
            print sumv
#                                       start=self._entry, stop=self._entry+1, cache_size=400000000)

            graph = ROOT.TGraph(len(regression.coef_), array.array('d', range(1,28,2)), array.array('d', regression.coef_))
            graph.SetMarkerStyle(7)
            graph.SetMarkerColor(2)
            stuff.append(graph)
            draw([graph], labels=['test'], text=tree_name, options='A', min_y=0., max_y=2.)
        
            hplot.data = hplot.data.append({'sample': samples[0].type,
                                           'pu': samples[0].label,
                                            'tp': tp,
                                            'tp_sel': tp_sel,
                                            'gen_sel': gen_sel,
                                            'classtype': 'pippo',
                                            'histo': HWrapper(graph),},
                                            ignore_index=True)

            hplot.data = hplot.data.append({'sample': samples[0].type,
                                           'pu': samples[0].label,
                                           'tp': tp,
                                           'tp_sel': tp_sel,
                                           'gen_sel': gen_sel,
                                           'classtype': EnergyRes,
                                           'histo': HWrapper(hres),},
                                           ignore_index=True)

            

TP: HMvDR sel: Em, gen_sel: GENEtaBC
Calibration Name: HMvDR_Em_GENEtaBC_calib nclusters: 25226, score: 0.988716444486, coefficients:
[7.16, 2.03, 1.06, 0.75, 1.24, 0.96, 0.83, 1.15, 1.06, 0.87, 0.86, 0.93, 1.24, 2.59]
22.7288118601
TP: HMvDR sel: all, gen_sel: GENEtaBC
Calibration Name: HMvDR_all_GENEtaBC_calib nclusters: 25321, score: 0.988463528118, coefficients:
[7.11, 2.04, 1.06, 0.76, 1.24, 0.96, 0.83, 1.15, 1.06, 0.86, 0.87, 0.91, 1.29, 2.62]
22.729596734
TP: HMvDRshape sel: Em, gen_sel: GENEtaBC
Calibration Name: HMvDRshape_Em_GENEtaBC_calib nclusters: 24960, score: 0.982861152409, coefficients:
[15.9, 1.57, 0.98, 1.43, 0.98, 0.85, 0.93, 0.97, 1.17, 1.06, 0.79, 0.83, 1.12, 2.85]
31.4480927587
TP: HMvDRshape sel: all, gen_sel: GENEtaBC
Calibration Name: HMvDRshape_all_GENEtaBC_calib nclusters: 25054, score: 0.98262772091, coefficients:
[15.83, 1.56, 0.98, 1.43, 0.98, 0.85, 0.93, 0.98, 1.17, 1.04, 0.81, 0.81, 1.16, 2.89]
31.4274486899
TP: HMvDRshapeDr sel: Em, gen_sel: GENEtaBC
C

Warning in <TROOT::Append>: Replacing existing TH1: HMvDR_Em_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDR_all_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDRshape_Em_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDRshape_all_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDRshapeDr_Em_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDRshapeDr_all_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDRcylind5_Em_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDRcylind5_all_GENEtaBC_energyResVenergy (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: HMvDRcylin

In [88]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V9,PU0,HMvDR,Em,GENEtaBC,pippo,<__main__.HWrapper object at 0x11dcea9d0>
1,V9,PU0,HMvDR,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x11dd4bf50>
2,V9,PU0,HMvDR,all,GENEtaBC,pippo,<__main__.HWrapper object at 0x1b9550490>
3,V9,PU0,HMvDR,all,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x11dd4b890>
4,V9,PU0,HMvDRshape,Em,GENEtaBC,pippo,<__main__.HWrapper object at 0x11dd27bd0>
5,V9,PU0,HMvDRshape,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x10c787590>
6,V9,PU0,HMvDRshape,all,GENEtaBC,pippo,<__main__.HWrapper object at 0x1396dda50>
7,V9,PU0,HMvDRshape,all,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x1b95b3f50>
8,V9,PU0,HMvDRshapeDr,Em,GENEtaBC,pippo,<__main__.HWrapper object at 0x1396b8ed0>
9,V9,PU0,HMvDRshapeDr,Em,GENEtaBC,__main__.EnergyRes,<__main__.HWrapper object at 0x1b95b4790>


In [89]:
hplot.data.tp.unique()

array(['HMvDR', 'HMvDRshape', 'HMvDRshapeDr', 'HMvDRcylind5',
       'HMvDRcylind2p5', 'HMvDRshapeCalib', 'HMvDRshapeDrCalib',
       'HMvDRcylind5Calib', 'HMvDRcylind2p5Calib'], dtype=object)

In [90]:
hplot.labels_dict

{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'GEN': '',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaBCDHBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaBCDLBrem': '1.52 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5',
 'GENEtaBCHBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} >= 0.5',
 'GENEtaBCLBrem': '1.52 < |#eta^{GEN}| <= 2.4, f_{BREM} < 0.5',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENEtaDHBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5',
 'GENEtaDHBremPt10': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=10GeV',
 'GENEtaDHBremPt20': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=20GeV',
 'GENEtaDHBremPt30': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=30GeV',
 'GENEtaDHBremPt40': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} >= 0.5, p_{T}^{GEN}>=40GeV',
 'GENEtaDLBrem': '2.4 < |#eta^{GEN}| <= 2.8, f_{BREM} < 0.5'

In [91]:
hfd = hplot.get_histo('pippo', 'V9', ['PU0'], ['HMvDRcylind2p5'], 'Em', 'GENEtaBC')

print hfd[0]
print hfd[1]
print hfd[2]




[<ROOT.TGraph object ("Graph") at 0x7ff20414df80>]
['']
PU0, HM Cylinder 2.5cm, EGId, 1.52 < |#eta^{GEN}| <= 2.4


In [92]:
print hfd[2]


PU0, HM Cylinder 2.5cm, EGId, 1.52 < |#eta^{GEN}| <= 2.4


## Comparison of the calibration sets for different selections

In [114]:
for tp in tps:
#     for tp_sel in tp_select[tp]:
        for gen_sel in ['GENEtaBC']:
            hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all', 'Em'], gen_sel)
            draw(hsets, labels,  text=text, min_y=0., max_y=2.)

In [115]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5', 'HMvDRshape', 'HMvDRshapeDr'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=10., h_lines=[1.])

<ROOT.TCanvas object ("c_5dd150") at 0x7ff206a24b30>

In [116]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind5', 'HMvDRcylind5Calib','HMvDRcylind5Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

<ROOT.TCanvas object ("c_33fb23") at 0x7ff206a27230>

In [117]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind10', 'HMvDRcylind10Calib','HMvDRcylind10Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

No match found for: pu: ['PU0'], tp: ['HMvDRcylind10', 'HMvDRcylind10Calib', 'HMvDRcylind10Calib1'], tp_sel: Em, gen_sel: GENEtaBC, classtype: pippo


TypeError: object of type 'NoneType' has no len()

In [97]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRshape', 'HMvDRshapeCalib','HMvDRshapeCalib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

<ROOT.TCanvas object ("c_e14350") at 0x7ff2022cb640>

In [118]:

hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind2p5Calib','HMvDRcylind2p5Calib1'], 'Em', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

<ROOT.TCanvas object ("c_c09dd5") at 0x7ff204573090>

In [119]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'Em', 'GENEtaBCPt10')
draw(hsets, labels,  text=text, min_y=0., max_y=10., h_lines=[1.])

No match found for: pu: ['PU0'], tp: ['HMvDRcylind2p5', 'HMvDRcylind5', 'HMvDRcylind10'], tp_sel: Em, gen_sel: GENEtaBCPt10, classtype: pippo


TypeError: object of type 'NoneType' has no len()

In [120]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'Em', 'GENEtaBCPt40')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

No match found for: pu: ['PU0'], tp: ['HMvDRcylind2p5', 'HMvDRcylind5', 'HMvDRcylind10'], tp_sel: Em, gen_sel: GENEtaBCPt40, classtype: pippo


TypeError: object of type 'NoneType' has no len()

In [74]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GEN')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

No match found for: pu: ['PU0'], tp: ['HMvDRcylind2p5', 'HMvDRcylind5', 'HMvDRcylind10'], tp_sel: all, gen_sel: GEN, classtype: pippo


TypeError: object of type 'NoneType' has no len()

In [121]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

<ROOT.TCanvas object ("c_5ecafd") at 0x7ff2046dfb60>

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind2p5', 'HMvDRcylind5','HMvDRcylind10'], 'all', 'GENEtaBC')
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    for gen_sel in ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40']:
        hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all', 'Em'], gen_sel)
        draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['Em'], ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40'])
    draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
for tp in tps:
    hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], tp, ['all'], ['GENEtaBC', 'GENEtaBCPt10', 'GENEtaBCPt20', 'GENEtaBCPt40'])
    draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
hsets, labels, text = hplot.get_histo('pippo', sample, ['PU0'], ['HMvDRcylind10'], ['all', 'Em'], ['GENEtaBC'])
draw(hsets, labels,  text=text, min_y=0., max_y=2., h_lines=[1.])

In [ ]:
len([1.527]+[1.]*12+[1.98])

## Energy Resolution

In [76]:
def gaussfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()

#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', "gaus", -100, 100, 5)
#     fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y,max_value, rms_value)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, int = {}, max_value = {}, RMS = {}'.format(name, project_hist.Integral(), max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result

def gausstailfit(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, -100, 100, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result

def gausstailfit_pt(name, project_hist):
    
    max_bin = project_hist.GetMaximumBin()
    max_value = project_hist.GetBinCenter(max_bin)
    rms_value = project_hist.GetRMS()
    max_y = project_hist.GetMaximum()
#     print max_bin, max_value, rms_value
    def gausstail(x, p):
        #         // [Constant] * ROOT::Math::crystalball_function(x, [Alpha], [N], [Sigma], [Mean])
        return p[0] * ROOT.Math.crystalball_function(x[0], p[3], p[4], p[2], p[1])

    fitf = ROOT.TF1('gausstail', gausstail, 0, 2, 5)
    fitf.SetParNames('norm', 'mean', 'sigma', 'alpha', 'n')
#     fitf.FixParameter(0, 1.)
    fitf.SetParLimits(1, max_value-15.,max_value+15.)

    fitf.SetParameters(max_y, max_value, rms_value, 1, 1)
    draw([project_hist], labels=['fit'], text=name)
    stuff.append(fitf)
    project_hist.Draw("same")
#     c.Draw()
    print '   name: {}, y_max = {}, max_value = {}, RMS = {}'.format(name, max_y, max_value, rms_value)
    result = project_hist.Fit('gausstail','QERLS+')
    result.Print()
    print '   norm = {}, reso_mean = {}, reso_sigma = {}, alpha = {}, n = {}'.format(result.GetParams()[0], result.GetParams()[1], result.GetParams()[2], result.GetParams()[3], result.GetParams()[4])
#     func = project_hist.GetFunction("gaus")
    # print '   NDF = {}, chi2 = {}, prob = {}'.format(fitf.GetNDF(), fitf.GetChisquare(), fitf.GetProb())
    return result



In [77]:
def computeEResolution(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[2]/x_mean)
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h

def computeEResolutionMean(h2d_orig):
    h2d = h2d_orig.Clone()
    h2d.GetYaxis().SetRangeUser(-100, 100)
    x, y, ex_l, ex_h, ey_l, ey_h = [], [], [], [], [], []
    print '-----------------------'
    for x_bin_low, x_bin_high in [(3,6),(7,12),(13,23),(24,34),(35,49),(50,100)]:
        y_proj = h2d.ProjectionY(uuid.uuid4().hex[:6]+'_y', x_bin_low, x_bin_high)
        stuff.append(y_proj)
        x_low = h2d.GetXaxis().GetBinLowEdge(x_bin_low)
        x_high = h2d.GetXaxis().GetBinUpEdge(x_bin_high)
        print 'x_low: {} x_high: {}'.format(x_low, x_high)
        fit_result = gausstailfit(h2d_orig.GetName(), y_proj)

        h2d.SetAxisRange(x_low, x_high)
        x_mean = h2d.GetMean()
#         print 'mean: {}'.format(x_mean)
    #     x_value = h2d.GetXaxis().GetBinCenter(x_bin)
    #     x_err_min = x_value - h2d.GetXaxis().GetBinLowEdge(x_bin)
    #     x_err_plus = h2d.GetXaxis().GetBinUpEdge(x_bin) - x_value

        x.append(x_mean)
        ex_l.append(0)
        ex_h.append(0)

        y.append(fit_result.GetParams()[1])
        ey_l.append(0)
        ey_h.append(0)
    return x, y, ex_l, ex_h, ey_l, ey_h



In [122]:

hsets, labels, text = hplot.get_histo(EnergyRes, sample, ['PU0'], ['HMvDRcylind5', 'HMvDRcylind5Calib','HMvDRcylind5Calib1'], 'Em', 'GENEtaBC')
draw([his.h_energyResVenergy for his in hsets], labels,  text=text, options='COLZ', min_y=-100, max_y=100)

<ROOT.TCanvas object ("c_2f1254") at 0x7ff20828f850>

In [123]:
for tp in tps:
    for pu in ['PU0']:
        hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
#         print dir(hsets[0])
        draw([his.h_energyResVenergy_graph('sigmaEOE', '#sigma_{E}/E', computeEResolution) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


-----------------------
x_low: 20.0 x_high: 60.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 252.0, max_value = -3.5, RMS = 6.8621753237
   norm = 247.742855264, reso_mean = -4.39607556517, reso_sigma = 2.79081517345, alpha = 0.395175721007, n = 12867513.933
x_low: 60.0 x_high: 120.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 311.0, max_value = -5.5, RMS = 10.1348326357
   norm = 298.55017905, reso_mean = -4.20832766533, reso_sigma = 3.86779257028, alpha = 0.394228233949, n = 719002.308587
x_low: 120.0 x_high: 230.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 439.0, max_value = -5.5, RMS = 11.8171928977
   norm = 446.884573455, reso_mean = -4.54898384455, reso_sigma = 5.86027565207, alpha = 0.618196075305, n = 14.8642617974
x_low: 230.0 x_high: 340.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 277.0, max_value = -3.5, RMS = 11.4499814873
   norm = 276.715363517, reso_mean = -4.06463441496, reso_sigma = 7.60948644702, alpha = 0.77795349739, n = 161.8

   norm = 387.111905968, reso_mean = -4.1527754347, reso_sigma = 6.22066018077, alpha = 0.534997073309, n = 12.7588379275
x_low: 230.0 x_high: 340.0
   name: HMvDRshapeCalib_Em_GENEtaBC_energyResVenergy, y_max = 239.0, max_value = -5.5, RMS = 13.7740432746
   norm = 243.945845359, reso_mean = -3.14006936368, reso_sigma = 7.90291574868, alpha = 0.62914650109, n = 827.484275059
x_low: 340.0 x_high: 490.0
   name: HMvDRshapeCalib_Em_GENEtaBC_energyResVenergy, y_max = 104.0, max_value = -2.5, RMS = 13.1945270694
   norm = 103.91081476, reso_mean = -2.68095774099, reso_sigma = 9.72433366166, alpha = 0.865027919658, n = 2475090.93885
x_low: 490.0 x_high: 1000.0
   name: HMvDRshapeCalib_Em_GENEtaBC_energyResVenergy, y_max = 14.0, max_value = -2.5, RMS = 14.79366439
   norm = 7.4721350278, reso_mean = -1.12796934312, reso_sigma = 11.678465604, alpha = 0.962836445403, n = 2198540.14283
-----------------------
x_low: 20.0 x_high: 60.0
   name: HMvDRshapeDrCalib_Em_GENEtaBC_energyResVenergy, y_ma

In [124]:
for tp in tps:
    for pu in ['PU0']:
        hsets, labels, text = hplot.get_histo(EnergyRes, sample, pu, tp, ['Em'], 'GENEtaBC', debug=False)
#         print dir(hsets[0])
        draw([his.h_energyResVenergy_graph('meanEReso', '<E_{L1}-E_{GEN}>', computeEResolutionMean) for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)



-----------------------
x_low: 20.0 x_high: 60.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 252.0, max_value = -3.5, RMS = 6.8621753237
   norm = 247.742855264, reso_mean = -4.39607556517, reso_sigma = 2.79081517345, alpha = 0.395175721007, n = 12867513.933
x_low: 60.0 x_high: 120.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 311.0, max_value = -5.5, RMS = 10.1348326357
   norm = 298.55017905, reso_mean = -4.20832766533, reso_sigma = 3.86779257028, alpha = 0.394228233949, n = 719002.308587
x_low: 120.0 x_high: 230.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 439.0, max_value = -5.5, RMS = 11.8171928977
   norm = 446.884573455, reso_mean = -4.54898384455, reso_sigma = 5.86027565207, alpha = 0.618196075305, n = 14.8642617974
x_low: 230.0 x_high: 340.0
   name: HMvDR_Em_GENEtaBC_energyResVenergy, y_max = 277.0, max_value = -3.5, RMS = 11.4499814873
   norm = 276.715363517, reso_mean = -4.06463441496, reso_sigma = 7.60948644702, alpha = 0.77795349739, n = 161.8

   norm = 387.111905968, reso_mean = -4.1527754347, reso_sigma = 6.22066018077, alpha = 0.534997073309, n = 12.7588379275
x_low: 230.0 x_high: 340.0
   name: HMvDRshapeCalib_Em_GENEtaBC_energyResVenergy, y_max = 239.0, max_value = -5.5, RMS = 13.7740432746
   norm = 243.945845359, reso_mean = -3.14006936368, reso_sigma = 7.90291574868, alpha = 0.62914650109, n = 827.484275059
x_low: 340.0 x_high: 490.0
   name: HMvDRshapeCalib_Em_GENEtaBC_energyResVenergy, y_max = 104.0, max_value = -2.5, RMS = 13.1945270694
   norm = 103.91081476, reso_mean = -2.68095774099, reso_sigma = 9.72433366166, alpha = 0.865027919658, n = 2475090.93885
x_low: 490.0 x_high: 1000.0
   name: HMvDRshapeCalib_Em_GENEtaBC_energyResVenergy, y_max = 14.0, max_value = -2.5, RMS = 14.79366439
   norm = 7.4721350278, reso_mean = -1.12796934312, reso_sigma = 11.678465604, alpha = 0.962836445403, n = 2198540.14283
-----------------------
x_low: 20.0 x_high: 60.0
   name: HMvDRshapeDrCalib_Em_GENEtaBC_energyResVenergy, y_ma

In [128]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshape','HMvDRcylind2p5','HMvDRcylind5', 'HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-100, max_y=100)


<ROOT.TCanvas object ("c_796608") at 0x7ff20822db00>

In [129]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeCalib','HMvDRcylind2p5Calib','HMvDRcylind5Calib', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_meanEReso for his in hsets], labels, text=text, options='COLZ', min_y=-10, max_y=10)


<ROOT.TCanvas object ("c_781594") at 0x7ff206995840>

In [130]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshape','HMvDRcylind2p5','HMvDRcylind5', 'HMvDRshapeDr'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


<ROOT.TCanvas object ("c_ed87fa") at 0x7ff203738240>

In [131]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeCalib','HMvDRcylind2p5Calib','HMvDRcylind5Calib', 'HMvDRshapeDrCalib'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


<ROOT.TCanvas object ("c_af6306") at 0x7ff2037375e0>

In [85]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRshapeCalib1','HMvDRcylind2p5Calib1','HMvDRcylind5Calib1', 'HMvDRcylind10Calib1'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
    

No match found for: pu: PU0, tp: ['HMvDRshapeCalib1', 'HMvDRcylind2p5Calib1', 'HMvDRcylind5Calib1', 'HMvDRcylind10Calib1'], tp_sel: ['Em'], gen_sel: GENEtaBC, classtype: __main__.EnergyRes


TypeError: 'NoneType' object is not iterable

In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRfixedCalib1','HMvDRfixed'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)


In [ ]:
hsets, labels, text = hplot.get_histo(EnergyRes, sample, 'PU0', ['HMvDRcylind2p5','HMvDRcylind2p5Calib1'], ['Em'], 'GENEtaBC', debug=False)
draw([his.g_energyResVenergy_sigmaEOE for his in hsets], labels, text=text, options='COLZ', min_y=0., max_y=0.1)
